_한국어로 기계번역됨_


# LangGraph 플랫폼을 사용하여 CrewAI, AutoGen 및 기타 프레임워크를 배포하는 방법

[LangGraph 플랫폼](https://langchain-ai.github.io/langgraph/concepts/langgraph_platform/)은 에이전트를 배포하기 위한 인프라를 제공합니다. 이는 LangGraph와 완벽하게 통합되지만, 다른 프레임워크와도 함께 사용할 수 있습니다. 이를 작동하게 하려면 에이전트를 단일 LangGraph 노드로 래핑하고, 이를 전체 그래프로 설정하면 됩니다.

이렇게 하면 LangGraph 플랫폼에 배포할 수 있으며, [이점](https://langchain-ai.github.io/langgraph/concepts/langgraph_platform/)을 많이 누릴 수 있습니다. 수평 확장이 가능한 인프라, 버스트 작업을 처리할 수 있는 작업 대기열, 단기 기억을 지원하는 지속성 레이어, 그리고 장기 기억 지원을 제공합니다.

이 가이드에서는 AutoGen 에이전트를 사용하여 이 방법을 보여주지만, 이 방법은 CrewAI, LlamaIndex 등 다른 프레임워크에서 정의된 에이전트에도 적용될 수 있습니다.


## 설정


In [10]:
%pip install autogen langgraph


In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


## 오토젠 에이전트 정의

여기에서 우리의 오토젠 에이전트를 정의합니다. https://github.com/microsoft/autogen/blob/0.2/notebook/agentchat_web_info.ipynb에서 가져왔습니다.


In [ ]:
import autogen
import os

config_list = [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]

llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

autogen_agent = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "web",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    llm_config=llm_config,
    system_message="Reply TERMINATE if the task has been solved at full satisfaction. Otherwise, reply CONTINUE, or the reason why the task is not solved yet.",
)


## LangGraph로 감싸기

이제 AutoGen 에이전트를 하나의 LangGraph 노드로 감싸고, 이를 전체 그래프로 만듭니다. 여기서 가장 중요한 것은 노드의 입력 및 출력 스키마를 정의하는 것으로, 수동으로 배포할 경우 필요하므로 추가 작업은 아닙니다.


In [11]:
from langgraph.graph import StateGraph, MessagesState


def call_autogen_agent(state: MessagesState):
    last_message = state["messages"][-1]
    response = user_proxy.initiate_chat(autogen_agent, message=last_message.content)
    # get the final response from the agent
    content = response.chat_history[-1]["content"]
    return {"messages": {"role": "assistant", "content": content}}


graph = StateGraph(MessagesState)
graph.add_node(call_autogen_agent)
graph.set_entry_point("call_autogen_agent")
graph = graph.compile()


## LangGraph 플랫폼으로 배포하기

이제 LangGraph 플랫폼을 사용하여 일반적으로 배포할 수 있습니다. 더 자세한 내용은 [이 지침](https://langchain-ai.github.io/langgraph/concepts/deployment_options/)을 참조하세요.
